# Librerías

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten,Dense,Dropout,BatchNormalization,LSTM,Conv1D,MaxPool1D
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

plt.rcParams['figure.figsize'] = [8, 6] 

In [ ]:
largo_generacion = 625000 #Valor en bytes, en bits multiplicar por 8

# Data post procesada
Se crea el arreglo de muestras partiendo del archivo data_1bit.txt que contiene 5000000 de datos post procesados del QRNG en formato binario.

In [ ]:
muestras = np.zeros(5000000, dtype = 'int8')

f = open("data_1bit.txt", "r")
cadena = f.read()

temp = ''
contador = 0

for a in cadena:
    if a != ",":
        temp += a
    else:
        muestras[contador] = int(temp)
        temp = ''
        contador += 1

Se construye histograma de probabilidad de la secuencia de números aleatorios

In [ ]:
probabilidades = np.zeros(2)

for i in range(0, len(probabilidades)):
    prob = np.count_nonzero(muestras == i)/len(muestras)
    probabilidades[i] = prob

nums_1bit = [0,1]

plt.bar(nums_1bit, probabilidades, color = 'blue')
plt.axhline(0.5, color = 'red', label = 'y = ' + str(0.5))
plt.xlabel('Número de 1 bit')
plt.ylabel('Probabilidad')
plt.legend()
plt.show()

# Conjuntos de ML
Se construyen los conjuntos de entrenamiento y prueba o test. Como input, se utilizan secuencias cuya longitud es dada por la variable "longitud_input". Dichas secuencias se codifican con vectores one-hot. Por otro lado, el corrimiento es dado por "Salto". La variable "porcentaje" determina qué fracción de la muestra será para entrenar y cuanto se utilizará para el test final.

In [ ]:
longitud_input = 13
Salto = 1
porcentaje = 0.5

longitud_entrenamiento = int(porcentaje * largo_generacion  * 8 / Salto)
longitud_test = int((1 - porcentaje) * largo_generacion * 8 / Salto - (longitud_input + 1))

inputs_entrenamiento = np.zeros((longitud_entrenamiento, longitud_input, 2), dtype = 'int8')
outputs_entrenamiento = np.zeros((longitud_entrenamiento), dtype = 'int8')

inputs_test = np.zeros((longitud_test, longitud_input, 2), dtype = 'int8')
outputs_test = np.zeros((longitud_test), dtype = 'int8')

for i in range(0,longitud_entrenamiento):
    for j in range(0, longitud_input):
        inputs_entrenamiento[i,j,muestras[i * Salto + j]] = 1
    
    outputs_entrenamiento[i] = muestras[i*Salto + longitud_input]
    
for i in range(longitud_entrenamiento,longitud_entrenamiento + longitud_test):
    for j in range(0, longitud_input):
        inputs_test[i - longitud_entrenamiento,j,muestras[i * Salto + j]] = 1
    
    outputs_test[i - longitud_entrenamiento] = muestras[i*Salto + longitud_input]
    
print(inputs_entrenamiento.shape)
print(outputs_entrenamiento.shape)
print(inputs_test.shape)
print(outputs_test.shape)

# Modelo de red recurrente convolucional (RCNN)
Se construye el modelo de red con keras.

In [ ]:
input_data_shape = (longitud_input, 2)
activacion_conv = 'relu'
activacion_lstm = 'tanh'
activacion_oculta = 'relu'
activacion_output = 'sigmoid'

RCNN = Sequential()

RCNN.add(Conv1D(input_shape = input_data_shape, filters = 32, kernel_size = 5, activation = activacion_conv,
                padding = 'same', name = 'capa_conv'))
RCNN.add(MaxPool1D(pool_size = 2, name = 'capa_pooling'))
RCNN.add(BatchNormalization())
RCNN.add(LSTM(units = 64, activation = activacion_lstm, return_sequences = False, name = 'capa_LSTM'))
RCNN.add(BatchNormalization())
RCNN.add(Dropout(0.1))
RCNN.add(Dense(units = 32, activation = activacion_oculta, name = 'capa_oculta'))
RCNN.add(BatchNormalization())
RCNN.add(Dense(1, activation = activacion_output, name = 'capa_outputs'))

RCNN.summary()

# Entrenamiento
Se entrena el modelo de machine learning con el conjunto de entrenamiento. Algunos hiperparámetros como la tasa de entrenamiento y el número de epochs se establecen al principio de la celda. La función de costo es "binary_crossentropy", pues se trata de un problema de clasificación binaria. Adicionalmente, se utiliza un 10% de la data de entrenamiento como data de validación.

Cabe mencionar que se aplica un EarlyStopping, el cual detiene el entrenamiento cuando el costo de validación no mejora en 10 epochs consecutivos. Al final del entrenamiento, se guarda la red con los parámetros que minimizaron el costo de validación.

In [ ]:
tasa_entrenamiento = 0.0005
numero_epochs = 50
tamanio_minilote = 1024

optimizador = Adam(learning_rate = tasa_entrenamiento)
funcion_costo = 'binary_crossentropy'
metrica = ['accuracy']
callback = EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights=True)

RCNN.compile(optimizer = optimizador, loss = funcion_costo, metrics = metrica)
entrenamiento = RCNN.fit(inputs_entrenamiento, outputs_entrenamiento, batch_size = tamanio_minilote,
                        callbacks = [callback], validation_split= 0.1, epochs = numero_epochs)

# Gráficas
Se observa la evolución de la red a través de gráficas de costo y precisión tanto sobre el conjunto de entrenamiento como el conjunto de validación.

In [ ]:
plt.plot(entrenamiento.history['accuracy'], label = 'Entrenamiento')
plt.plot(entrenamiento.history['val_accuracy'], label = 'Validación')
plt.axhline(0.5, color = 'red', label = 'P. Adivinar')
plt.ylabel('Precisión')
plt.xlabel('Epoch')
plt.grid()
plt.legend()
plt.show()

plt.plot(entrenamiento.history['loss'], label = 'Entrenamiento')
plt.plot(entrenamiento.history['val_loss'], label = 'Validación')
plt.ylabel('Costo')
plt.xlabel('Epoch')
plt.grid()
plt.legend()
plt.show()

# Evaluación
Se evalúa la red mediante el conjunto de prueba o test para observar si el modelo pudo generalizarse.

In [ ]:
costo_test, precision_test = RCNN.evaluate(inputs_test, outputs_test)

print("El costo sobre el conjunto de prueba es: " + str(costo_test))
print("La precisión sobre el conjunto de prueba es: " + str(precision_test))